In [97]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [98]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [99]:
USER = Path(os.path.expanduser("~"))

In [100]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [101]:
dados_cotacao__aj_consolidado_excel

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,2024-02-15 00:00:00,"277,7","124,06","12,38","226,102418805","4,8",127804,"103,28",8778,15414,...,"92,8976217489","8,91286447742","87,1138471971","82,4769332469","8,7287315473","8,59162617668","11,0362010252","105,625387195","113,849902134","22,2779974546"
1,16/02/2024,"276,35","124,88","12,24",NaN,"4,88",128725,"104,01",8845,15332,...,"92,9905472483","8,94130978958","86,6538710642","82,4769332469","8,69082174688","8,59162617668","11,2518617079","105,798669821","113,353657799","22,2117502234"
2,19/02/2024,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
3,20/02/2024,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
4,21/02/2024,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,06/08/2024,"329,72","122,74","14,24",NaN,"5,81",126266,"98,4",8868,18481,...,"96,35","9,23","83,86","70,23","9,96","9,02","9,27","101,21","111,3","22,24"
121,07/08/2024,"325,81","124,04",14,NaN,"5,8",127513,"100,9",8882,18254,...,"96,11","9,28",83,"70,22","9,76","9,02","9,3","101,1","111,93","22,57"
122,08/08/2024,"330,72","125,3","14,29",NaN,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
123,09/08/2024,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"


In [102]:
dados_cotacao__aj_consolidado_excel['Data'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['Data'], format="%d/%m/%Y")

In [103]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['Data'].last_valid_index()

In [104]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_1=(datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")
data_fim_1=(datetime.now() - timedelta(days=120)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_1}%26data_fim%3D{data_fim_1}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_1 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [105]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_2=(datetime.now() - timedelta(days=119)).strftime("%d/%m/%Y")
data_fim_2=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_2}%26data_fim%3D{data_fim_2}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_2 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [106]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_3=(datetime.now() - timedelta(days=59)).strftime("%d/%m/%Y")
data_fim_3=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_3}%26data_fim%3D{data_fim_3}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_3 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [107]:
data_1 = list()
for lin in (table := response_1.json()["tables"]["tab0"]):
    data_1.append(table[lin])

In [108]:
dados_cotacao__aj_novos_1 = pd.DataFrame(data_1)

In [109]:
data_2 = list()
for lin in (table := response_2.json()["tables"]["tab0"]):
    data_2.append(table[lin])

In [110]:
dados_cotacao__aj_novos_2 = pd.DataFrame(data_2)

dados_cotacao__aj_novos_2 = dados_cotacao__aj_novos_2.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_2.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,16/04/2024,"295,81","120,85","13,01","212,578162964","5,17",124388,"100,45",8626,16568,...,"96,375589735","9,09244454504","89,2311194218","71,6363027313","9,53341042855","8,77573685306","9,43784411434","104,469697704","114,610499503","20,8962694901"
1,17/04/2024,"292,4","120,54","12,79","205,835982324","5,18",124171,"99,85",8647,16375,...,"96,7114266168","9,11181077411","89,5026333692","71,2202064167","9,59183786734","8,7564918161","9,32192381815","104,323244857","114,019424049","20,6596722359"
2,18/04/2024,"292,3","120,48","12,66",,"5,24",124196,"99,07",8645,16287,...,"96,4331617719","9,10212765957","89,5220272225","70,9492599792","9,57236205441","8,74686929762","9,38954399093","103,786251084","113,360520264","20,4893222129"
3,19/04/2024,287,"121,52","12,36","198,954170724","5,18",125124,101,8734,15859,...,"97,0088821407","9,11181077411","89,696571903","70,5234870061","9,57236205441","8,78535937154","9,4571641637","103,669088806","113,27331241","20,5461055539"
4,22/04/2024,"288,29","121,8","12,39","202,963574004","5,25",125573,"101,11",8762,15968,...,"96,855356709","9,11181077411","89,5026333692","71,5105061711","9,61131368027","8,73724677914","9,37988396624","103,581217098","111,713260802","20,5650333342"
5,23/04/2024,"289,7","121,41","12,52","193,708036084","5,27",125148,"100,9",8667,16081,...,"96,9129287459","9,07307831598","88,435971433","71,2589130506","9,61131368027","8,73724677914","9,36056391688","103,493345389","111,529155333","20,6596722359"
6,24/04/2024,"289,73","120,97","12,58","187,763746644","5,33",124740,"99,98",8618,16012,...,"96,9033334064","9,11181077411","88,2032451924","69,6816177183","9,61131368027","8,68913418674","9,17702344792","102,321722611","110,996218448","20,4420027621"
7,25/04/2024,"290,51","120,91","12,54","188,362165044","5,42",124645,"99,4",8578,16264,...,"97,6325792068","9,08276143051","87,4177941303","70,2428639101","9,61131368027","8,66026663131","9,14804337388","102,272904996","110,124139909","19,4388304043"
8,26/04/2024,"289,8","122,83","12,62",,"5,46",126526,"101,76",8673,16200,...,"97,1048355355","9,11181077411","87,7087019311","70,1267440084","9,56262414794","8,64102159435","8,98382295428","102,243614426","110,928390117","20,3000444096"
9,29/04/2024,"290,95","123,77","12,67","188,182639524","5,52",127351,"102,85",8735,16257,...,"97,2391702882","9,12149388864","88,435971433","69,7590309861","9,54314833502","8,69875670522","8,94518285555","102,22408738","111,82953794","20,3473638604"


In [111]:
data_3 = list()
for lin in (table := response_3.json()["tables"]["tab0"]):
    data_3.append(table[lin])

In [112]:
dados_cotacao__aj_novos_3 = pd.DataFrame(data_3)

dados_cotacao__aj_novos_3 = dados_cotacao__aj_novos_3.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_3.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,17/06/2024,"330,1","115,74","15,06",,"5,96",119137,93,8400,19171,...,"95,6283336288","9,18021505376","87,0526819709","66,595604067","9,7","8,82817074861","9,26385169928","100,975521971","108,812430174","20,65"
1,18/06/2024,"331,93","116,12","15,1",,"5,98",119630,"93,3",8463,19264,...,"95,7165154938","9,2","87,4493225084","64,7154904306","9,8","8,82817074861","9,26385169928","100,985369398","108,841955978","20,59"
2,19/06/2024,"332,75","116,76","15,23",,"5,94",120261,"93,49",8510,19432,...,"95,7263134788","9,2","87,4096584546","65,3092105263","9,76","8,84778890583","9,21441812564","100,768725993","109,934410731","20,61"
3,20/06/2024,"332,68","117,12","15,05","202,990265392","6,02",120445,94,8560,19254,...,"95,3931819887","9,27913978495","87,9451231803","65,6060705742","9,74","8,81836167","9,26385169928","99,8529152315","109,835991384","20,64"
4,21/06/2024,"330,85","118,02","14,95","199,952805229","5,94",121341,"94,39",8623,19140,...,"95,9810610889","9,21978494624","87,4790705487","65,2993151914","9,73","8,79874351278","9,36271884655","95,5298945425","109,058478542","20,83"
5,24/06/2024,327,"119,08","14,62",,"5,93",122636,"96,3",8712,18806,...,"95,2756061687","9,22967741935","87,6476427772","65,8138726077","9,66","8,82817074861","9,382492276","95,8154699411","108,910849521","21,3"
6,25/06/2024,332,"118,9","14,97",,"5,92",122331,96,8699,19208,...,"95,1384343786","9,20989247312","87,0130179171","65,0420364833","9,6","8,85759798444","9,47147270855","96,8691446876","110,229668773","21,28"
7,26/06/2024,"336,16","119,14","15,17",,6,122641,"96,12",8711,19489,...,"94,7955049035","9,25935483871","86,9733538634","65,6753379187","9,77","8,84778890583","9,3429454171","97,8046503037","112,286633128",21
8,27/06/2024,"336,25","120,8","15,18",,"5,86",124307,"97,8",8783,19477,...,"96,382778474","9,20989247312","87,3104983203","65,9820933014","9,79","8,84778890583","9,61977342945","98,2576319704","111,528804155","21,24"
9,28/06/2024,"339,83","120,38","15,32","210,01439202","5,96",123906,"96,45",8764,19721,...,"97,4899507793","9,22967741935","89,2242889138","66,18","9,85","8,88702522026","9,6","98,3561062458","109,855675254","21,12"


In [113]:
dados_cotacao__aj_novos = pd.concat([dados_cotacao__aj_novos_1, dados_cotacao__aj_novos_2, dados_cotacao__aj_novos_3], ignore_index=True) 

In [114]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [115]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [116]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [117]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,15/02/2024,"277,7","124,06","12,38","226,102418805","4,8",127804,"103,28",8778,15414,...,"92,8976217489","8,91286447742","87,1138471971","82,4769332469","8,7287315473","8,59162617668","11,0362010252","105,625387195","113,849902134","22,2779974546"
2,16/02/2024,"276,35","124,88","12,24",,"4,88",128725,"104,01",8845,15332,...,"92,9905472483","8,94130978958","86,6538710642","82,4769332469","8,69082174688","8,59162617668","11,2518617079","105,798669821","113,353657799","22,2117502234"
3,19/02/2024,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
4,20/02/2024,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
5,21/02/2024,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,07/08/2024,"325,81","124,04",14,,"5,8",127513,"100,9",8882,18254,...,"96,11","9,28",83,"70,22","9,76","9,02","9,3","101,1","111,93","22,57"
123,08/08/2024,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
124,09/08/2024,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
125,12/08/2024,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"


In [118]:
dados_cotacao__aj_novos.reset_index(drop = True)

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,15/02/2024,"277,7","124,06","12,38","226,102418805","4,8",127804,"103,28",8778,15414,...,"92,8976217489","8,91286447742","87,1138471971","82,4769332469","8,7287315473","8,59162617668","11,0362010252","105,625387195","113,849902134","22,2779974546"
1,16/02/2024,"276,35","124,88","12,24",,"4,88",128725,"104,01",8845,15332,...,"92,9905472483","8,94130978958","86,6538710642","82,4769332469","8,69082174688","8,59162617668","11,2518617079","105,798669821","113,353657799","22,2117502234"
2,19/02/2024,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
3,20/02/2024,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
4,21/02/2024,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,07/08/2024,"325,81","124,04",14,,"5,8",127513,"100,9",8882,18254,...,"96,11","9,28",83,"70,22","9,76","9,02","9,3","101,1","111,93","22,57"
122,08/08/2024,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
123,09/08/2024,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
124,12/08/2024,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"


In [119]:
dados_cotacao__aj_novos['Data'] = pd.to_datetime(dados_cotacao__aj_novos['Data'])

C:\Users\GiordanoBrunoGava\AppData\Local\Temp\ipykernel_5200\1604425695.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dados_cotacao__aj_novos['Data'] = pd.to_datetime(dados_cotacao__aj_novos['Data'])


In [120]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,2024-02-15,"277,7","124,06","12,38","226,102418805","4,8",127804,"103,28",8778,15414,...,"92,8976217489","8,91286447742","87,1138471971","82,4769332469","8,7287315473","8,59162617668","11,0362010252","105,625387195","113,849902134","22,2779974546"
2,2024-02-16,"276,35","124,88","12,24",,"4,88",128725,"104,01",8845,15332,...,"92,9905472483","8,94130978958","86,6538710642","82,4769332469","8,69082174688","8,59162617668","11,2518617079","105,798669821","113,353657799","22,2117502234"
3,2024-02-19,"276,7","125,14","12,16","229,772718325","4,98",129035,"103,73",8876,15341,...,"93,1206429475","8,9507915603","86,1888574094","82,4674857174","8,74768644751","8,59162617668","11,7206892791","107,146423577","113,076906151","22,3915641366"
4,2024-02-20,"272,3","126,19","12,05","226,122366085","4,8",129916,"105,85",8892,15063,...,"93,7618288933","8,9507915603","86,7392817355","82,4769332469","8,74768644751","8,61050887157","11,3044581828","107,541122891","112,924215586","22,694408622"
5,2024-02-21,"273,8","126,18","12,04","226,790599965","4,9",130031,"105,05",8847,15032,...,"93,808291643","9,06457280895","87,441547255","82,4769332469","8,7287315473","8,60106752413","11,4936541357","107,820300455","113,468175722","22,6849447318"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2024-08-07,"325,81","124,04",14,,"5,8",127513,"100,9",8882,18254,...,"96,11","9,28",83,"70,22","9,76","9,02","9,3","101,1","111,93","22,57"
123,2024-08-08,"330,72","125,3","14,29",,"5,88",128660,"102,11",8942,18593,...,96,"9,21","82,88","70,23","9,85","9,03","9,46",101,"110,31","22,63"
124,2024-08-09,"327,9","127,13","14,2","230,31","5,8",130614,"104,2",9039,18556,...,"95,76","9,21","83,28","68,25","9,82","9,01","9,31","102,03","111,7","23,07"
125,2024-08-12,"327,45","127,65","14,2",,"5,83",131115,104,9093,18505,...,96,"9,17","84,4","68,95","9,75","9,04","9,32","102,3","112,19","23,05"


In [121]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [122]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [123]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D15/06/2024%26data_fim%3D13/08/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
